In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import community.community_louvain as community

In [44]:
# load data
city_name = 'Sicklerville'
nodes_edges_name = 'mixed'

nodes = pd.read_csv(f'nodes_and_edges/{city_name}_{nodes_edges_name}_nodes.csv')
edges = pd.read_csv(f'nodes_and_edges/{city_name}_{nodes_edges_name}_edges.csv')

In [45]:
G = nx.Graph()

num_of_alone_nodes = 0
for _, row in nodes.iterrows():
    if row['id'] in edges['from'].values or row['id'] in edges['to'].values:
        G.add_node(row['id'], name=row['people_name'])
    else:
        num_of_alone_nodes += 1

print(f"Number of nodes that are not connected to any other node: {num_of_alone_nodes}")

for _, row in edges.iterrows():
    G.add_edge(row['from'], row['to'], weight=row['weight'], type=row['type'])

partition = community.best_partition(G)

Number of nodes that are not connected to any other node: 2200


In [46]:
"""palette = plt.get_cmap('tab20')
plt.figure(figsize=(10, 6))
pos = nx.spring_layout(G)
nx.draw(G, pos,
        node_color=[palette(partition[node]) for node in G.nodes()],
        node_size=50,
        edge_color='black',
        edgecolors='black',
        width=1,
        with_labels=False)

legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=palette(i), markersize=10, label=str(i)) 
                  for i in range(len(set(partition.values())))]
plt.legend(handles=legend_handles, title='Community', bbox_to_anchor=(1, 1), loc='upper left')

plt.title(f"{city_name} {nodes_edges_name} Network (Communities)")
plt.show()"""

# use plotly for better visualization
import plotly.graph_objects as go

pos = nx.spring_layout(G)
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='#888'), hoverinfo='none', mode='lines')

node_x = []
node_y = []
node_text = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(G.nodes[node]['name'])
    
node_trace = go.Scatter(x=node_x, y=node_y, mode='markers', hoverinfo='text', marker=dict(showscale=False, colorscale='YlGnBu', color=[], size=10, line_width=2))

for i, comm in enumerate(set(partition.values())):
    node_trace.marker.color += tuple([i] * list(partition.values()).count(comm))

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(title=f"{city_name} {nodes_edges_name} Network (Communities)", title_x=0.5, showlegend=False, hovermode='closest', margin=dict(b=20, l=5, r=5, t=40), xaxis=dict(showgrid=False, zeroline=False, showticklabels=False), yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

fig.show()

In [47]:
# Assortativity
